In [1]:
import warnings
warnings.filterwarnings('ignore')

# Inception

InceptionV3 est un modèle de réseau neuronal convolutif (CNN) qui a été formé sur plus d’un million d’images de la base de données ImageNet. Il est conçu par Google et décrit dans le document ‘Rethinking the Inception Architecture for Computer Vision’.

## Les concepts d'InceptionV3

*Architecture Inception* : L’idée principale derrière l’architecture Inception est de trouver comment un réseau de neurones optimal peut être construit de manière automatique. Pour ce faire, Inception utilise une approche modulaire, où un bloc Inception (ou module) est créé et répété dans le réseau. Chaque bloc Inception contient plusieurs filtres de convolution de différentes tailles qui sont exécutés en parallèle.


*Facteur de normalisation* : InceptionV3 utilise la normalisation par lots, qui est une technique pour améliorer la vitesse, les performances et la stabilité des réseaux de neurones artificiels. Elle normalise les couches d’entrée en réajustant et en redimensionnant les activations.


*Convolution asymétrique* : Pour réduire le nombre de paramètres et le coût de calcul, InceptionV3 utilise une convolution asymétrique, c’est-à-dire qu’elle remplace les grands filtres de convolution (par exemple, 5x5) par deux filtres de convolution plus petits (3x3 puis 3x1).


*Décomposition en profondeur* : InceptionV3 décompose les filtres de convolution 7x7 en filtres 3x3. Cela réduit le nombre de paramètres et améliore l’efficacité du réseau.
Global Average Pooling : À la fin du réseau, une couche de Global Average Pooling est utilisée à la place des couches entièrement connectées. Cela réduit considérablement le nombre de paramètres dans le réseau et aide à lutter contre le surapprentissage.


*Softmax* : Enfin, une couche softmax est utilisée pour produire une distribution de probabilité pour les prédictions de classe.


## Exemple

### Tensorflow

In [12]:
# Importation des bibliothèques nécessaires
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

# Téléchargement du modèle InceptionV3 pré-entraîné
model = InceptionV3(weights='imagenet')

# Fonction pour prédire la classe d'une image
def predict_class(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    print('Prédiction :', decode_predictions(preds, top=1)[0][0][1:3])


predict_class('../assets/car.jpg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 719ms/step
Prédiction : ('sports_car', 0.6899679)


### Pytorch

In [14]:
# Importation des bibliothèques nécessaires
import torch
from torchvision import models, transforms
from PIL import Image

# Chargement des noms de classes ImageNet
with open('../assets/imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

# Téléchargement du modèle InceptionV3 pré-entraîné
model = models.inception_v3(pretrained=True)
model.eval()

# Définition de la transformation de l'image
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Fonction pour prédire la classe d'une image
def predict_class(image_path):
    img = Image.open(image_path)
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)

    with torch.no_grad():
        out = model(batch_t)
    _, indices = torch.max(out, 1)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    print('Prédiction :', classes[indices[0]], percentage[indices[0]].item())

# Test de la fonction de prédiction
predict_class('../assets/car.jpg')


/Users/ben/miniconda3/envs/ai-explorer/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ben/miniconda3/envs/ai-explorer/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Prédiction : sports car 80.13818359375


## Quand utiliser le modele Inception ? 

*Classification d’images* : Inception est un excellent choix pour les tâches de classification d’images en raison de sa précision élevée. Il a été formé sur la base de données ImageNet, qui contient plus d’un million d’images de 1000 classes différentes.


*Transfert d’apprentissage* : Inception est souvent utilisé comme point de départ pour le transfert d’apprentissage. Le transfert d’apprentissage est une technique où un modèle pré-entraîné est utilisé sur un nouveau problème similaire. Il est particulièrement utile lorsque vous disposez de peu de données pour votre problème spécifique.


*Extraction de caractéristiques* : Les modèles Inception peuvent être utilisés pour extraire des caractéristiques utiles à partir d’images. Ces caractéristiques peuvent ensuite être utilisées pour former d’autres modèles ou pour effectuer d’autres tâches, comme la recherche d’images similaires.

*Fine-tuning* : Si vous avez suffisamment de données et de ressources de calcul, vous pouvez effectuer un fine-tuning du modèle Inception sur vos propres données. Cela peut améliorer les performances si vos données sont suffisamment différentes de celles sur lesquelles Inception a été initialement formé.


Il est important de noter que bien que Inception soit un modèle puissant, il n’est pas toujours le meilleur choix pour chaque tâche. Par exemple, pour les tâches impliquant des images de très haute résolution, des modèles avec une architecture différente peuvent être plus appropriés. Il peut egalement être trop complexe et coûteux en calcul pour certaines applications, en particulier celles qui nécessitent une latence faible ou qui sont déployées sur des appareils avec des ressources limitées. Dans ces cas, des modèles plus petits et plus efficaces, comme MobileNet, peuvent être une meilleure option.